### Our Mission 

Spam detection is one of the major applications of Machine Learning in the interwebs today. Pretty much all of the major email service providers have spam detection systems built in and automatically classify such mail as 'Junk Mail'. 

In this mission we will be using the Naive Bayes algorithm to create a model that can classify SMS messages as spam or not spam, based on the training we give to the model. It is important to have some level of intuition as to what a spammy text message might look like. Often they have words like 'free', 'win', 'winner', 'cash', 'prize' and the like in them as these texts are designed to catch your eye and in some sense tempt you to open them. Also, spam messages tend to have words written in all capitals and also tend to use a lot of exclamation marks. To the human recipient, it is usually pretty straightforward to identify a spam text and our objective here is to train a model to do that for us!

Being able to identify spam messages is a `binary classification` problem as messages are classified as either 'Spam' or 'Not Spam' and nothing else. Also, this is a supervised learning problem, as we will be feeding a labelled dataset into the model, that it can learn from, to make future predictions. 

#### 1. Observing and Loading the dataset:

Do refer to the following dataset that we will use: [SMSSpamClassifier](http://localhost:8888/edit/Workspace/Notebooks/Supervised%20Learning/Naive%20Bayes/data/SMSSpamCollection)

 - The columns in the dataset are currently not named and as you can see, there are 2 columns. 
 - The first column takes two values, `'ham'` which signifies that the message is not spam, and `'spam'` which signifies that the message is spam. 
 - The second column is the `text content` of the SMS message that is being classified.

In [1]:
# Import the dataset into a pandas dataframe using the `read_table` method.  
# Because this is a tab separated dataset we will be using '\t' as the value for the 'sep' argument which specifies 
# this format.
# Rename the column names by specifying a list ['label', 'sms_message'] to the 'names' argument of read_table().

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sms_dump = pd.read_table('./data/SMSSpamCollection', sep = '\t', names = ['Label', 'sms_message'])
sms_dump.head()

,Label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#### 2. Preprocess the data:

Now that we have a basic understanding of what our dataset looks like, lets preprocess our data:
 - Convert the 'Label' values into binary values - '0' to represent 'ham' and '1' to represent 'spam'.
 - Use the 'Bag of Words' concept to transform our text data into a word frequency table.

First, we will convert the labels:

In [2]:
# Use the pd.map method to map the binary values to the existing labels using a dictionary

sms_dump['Label'] = sms_dump['Label'].map({'ham': 0, 'spam': 1})
sms_dump.head()

,Label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms_dump.shape

(5572, 2)

Now using the 'Bag of Words' concept, transform the sms text data to a word frequency table.

Bag of Words (BoW) concept which is a term used to specify the problems that have a 'bag of words' or a collection of text data that needs to be worked with. The basic idea of BoW is to take a piece of text and count the frequency of the words in that text. It is important to note that the BoW concept treats each word individually and the order in which the words occur does not matter.

We can convert a collection of documents to a matrix, with each document being a row and each word (token) being the column, and the corresponding (row, column) values being the frequency of occurrence of each word or token in that document.

To handle this, we will be using sklearn's 
[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) method which does the following:

* It tokenizes the string (separates the string into individual words) and gives an integer ID to each token.
* It counts the occurrence of each of those tokens.

**Please Note:** 

* The CountVectorizer method automatically converts all tokenized words to their lower case form so that it does not treat words like 'He' and 'he' differently. It does this using the `lowercase` parameter which is by default set to `True`.

* It also ignores all punctuation so that words followed by a punctuation mark (for example: 'hello!') are not treated differently than the same words not prefixed or suffixed by a punctuation mark (for example: 'hello'). It does this using the `token_pattern` parameter which has a default regular expression which selects tokens of 2 or more alphanumeric characters.

* The third parameter to take note of is the `stop_words` parameter. Stop words refer to the most commonly used words in a language. They include words like 'am', 'an', 'and', 'the', etc. By setting this parameter value to `english`, CountVectorizer will automatically ignore all words (from our input text) that are found in the built in list of English stop words in scikit-learn. This is extremely helpful as stop words can skew our calculations when we are trying to find certain key words that are indicative of spam.

In [5]:
# Before using CountVectorizer on our data, split the dataset into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sms_dump['sms_message'], sms_dump['Label'], test_size = 0.3,
                                                   random_state = 1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3900,), (1672,), (3900,), (1672,))

Now that we have split the data, our next objective is to convert our data into the desired matrix format. To do this we will be using CountVectorizer(). There are two  steps to consider here:

* First, we have to fit our training data (`X_train`) into `CountVectorizer()` and return the matrix.
* Secondly, we have to transform our testing data (`X_test`) to return the matrix. 

Note that `X_train` is our training data for the 'sms_message' column in our dataset and we will be using this to train our model. 

`X_test` is our testing data for the 'sms_message' column and this is the data we will be using (after transformation to a matrix) to make predictions on. We will then compare those predictions with `y_test` in a later step. 

In [8]:
'''
The code for this segment is in 2 parts. First, we are learning a vocabulary dictionary for the training data 
and then transforming the data into a document-term matrix; secondly, for the testing data we are only 
transforming the data into a document-term matrix.
'''

from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

training_data = count_vector.fit_transform(X_train)

test_data = count_vector.transform(X_test)

training_data.shape, test_data.shape

((3900, 7155), (1672, 7155))

#### 3. Fit the preprocessed data and make predictions:

Now, since preprocessing is done, we will use them to train our model and make predictions. sklearn has several Naive Bayes implementations that we can use, so we do not have to do the math from scratch. We will be using sklearn's `sklearn.naive_bayes` method to make predictions on our SMS messages dataset.

Specifically, we will be using the `multinomial Naive Bayes algorithm`. This particular classifier is suitable for classification with discrete features (such as in our case, word counts for text classification). It takes in integer word counts as its input. On the other hand, `Gaussian Naive Bayes` is better suited for continuous data as it assumes that the input data has a Gaussian (normal) distribution.

In [9]:
'''
We have loaded the training data into the variable 'training_data' and the testing data into the 
variable 'testing_data'.

Import the MultinomialNB classifier and fit the training data into the classifier using fit(). Name your classifier
'naive_bayes'. You will be training the classifier using 'training_data' and 'y_train' from our split earlier. 
'''

from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB()

naive_bayes.fit(training_data, y_train)

MultinomialNB()

In [10]:
# Predict using the 'test_data' and store it into the 'predictions' variable

predictions = naive_bayes.predict(test_data)
predictions

array([0, 0, 0, ..., 0, 0, 0])

#### 4. Evaluating the model:

Now that we have made predictions on our test set, our next goal is to evaluate how well our model is doing. There are various mechanisms for doing so, so first let's review them.

 - **Accuracy** measures how often the classifier makes the correct prediction. It’s the ratio of the number of correct predictions to the total number of predictions (the number of test data points).

 - **Precision** tells us what proportion of messages we classified as spam, actually were spam.
    It is a ratio of true positives (words classified as spam, and which actually are spam) to all positives (all words classified as spam, regardless of whether that was the correct classification). In other words, precision is the ratio of:-

    `[True Positives/(True Positives + False Positives)]`
    

 - **Recall (sensitivity)** tells us what proportion of messages that actually were spam were classified by us as spam.
    It is a ratio of true positives (words classified as spam, and which actually are spam) to all the words that were actually spam. In other words, recall is the ratio of:-

    `[True Positives/(True Positives + False Negatives)]`
    

 - **F1-Score:** For classification problems that are skewed in their classification distributions like in our case - for example if we had 100 text messages and only 2 were spam and the other 98 weren't - accuracy by itself is not a very good metric. We could classify 90 messages as not spam (including the 2 that were spam but we classify them as not spam, hence they would be false negatives) and 10 as spam (all 10 false positives) and still get a reasonably good accuracy score. For such cases, `precision and recall` come in very handy. These two metrics can be combined to get the **F1-Score**, which is the weighted average of the precision and recall scores. This score can range from 0 to 1, with 1 being the best possible F1-Score.
 
 
We will be using all 4 of these metrics to make sure our model does well. For all 4 metrics whose values can range from 0 to 1, having a score as close to 1 as possible is a good indicator of how well our model is doing:

In [12]:
'''
Compute the accuracy, precision, recall and F1-Scores of your model using your test data 'y_test' and the predictions
you made earlier stored in the 'predictions' variable.
'''

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions)))
print('Recall score: ', format(recall_score(y_test, predictions)))
print('F1 score: ', format(f1_score(y_test, predictions)))

Accuracy score:  0.9874401913875598
Precision score:  0.9728506787330317
Recall score:  0.9347826086956522
F1 score:  0.9534368070953437


#### 5. Conclusion

One of the major advantages that Naive Bayes has over other classification algorithms is its ability to handle an extremely large number of features. In our case, each word is treated as a feature and there are thousands of different words. Also, it performs well even with the presence of irrelevant features and is relatively unaffected by them. The other major advantage it has is its relative simplicity. Naive Bayes' works well right out of the box and tuning its parameters is rarely ever necessary, except usually in cases where the distribution of the data is known. 
It rarely ever overfits the data. Another important advantage is that its model training and prediction times are very fast for the amount of data it can handle. All in all, Naive Bayes' really is a gem of an algorithm!